In [4]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import accuracy
from sklearn.metrics import mean_absolute_error
from surprise.model_selection import train_test_split
from sigopt import Connection
import sigopt

# Load the movielens-100k dataset (download it if needed),
data = Dataset.load_builtin(name='ml-100k', prompt=False)

# Or if you'd rather, comment the above line and uncomment the below line
# for a larger data set, while anticipating longer training times.

# data = Dataset.load_builtin(name='ml-1M', prompt=False)

# We'll use the famous SVD algorithm.
algo = SVD()

# Run 5-fold cross-validation and print results
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# sample random trainset and testset
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=.25)

# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)


from surprise.model_selection import GridSearchCV
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])


#!pip install sigopt
# Pass your API token directly, overriding any environment variables


# Замініть 'YOUR_API_TOKEN' на ваш фактичний API-токен SigOpt
conn = sigopt.Connection(client_token='WYKGPMSPGMXTRNJMHSNLQYXOYEJMBUXTYQDZVAKWZNVOTLQO')

experiment = conn.experiments().create(
    name='SVD',
    parameters=[
        dict(name='n_epochs', type='int', bounds=dict(min=5, max=10)),
        dict(name='lr_all', type='double', bounds=dict(min=0.002, max=0.005)),
        dict(name='reg_all', type='double', bounds=dict(min=0.4, max=0.6))
    ],
    metrics=[
        dict(name='RMSE', objective='minimize', strategy='optimize'),
        dict(name='MAE', objective='minimize', strategy='store')
    ],
    parallel_bandwidth=1,
    observation_budget=50,
)

print("Created experiment: https://app.sigopt.com/experiment/" + experiment.id)


# Evaluate your model with the suggested parameter assignments
def create_model(assignments):
  algo = SVD(
      n_epochs=assignments['n_epochs'],
      lr_all=assignments['lr_all'],
      reg_all=assignments['reg_all']
  ).fit(trainset)
  return algo

def evaluate_model(assignments):
  algo = create_model(assignments)
  predictions = algo.test(testset)
  # Then compute RMSE and MAE:
  return [
      dict(name="RMSE", value=accuracy.rmse(predictions)),
      dict(name="MAE", value=accuracy.mae(predictions))
    ]

# Run the Optimization Loop until the Observation Budget is exhausted
while experiment.progress.observation_count < experiment.observation_budget:
  suggestion = conn.experiments(experiment.id).suggestions().create()
  value_dicts = evaluate_model(suggestion.assignments)
  conn.experiments(experiment.id).observations().create(
    suggestion=suggestion.id,
    values=value_dicts,
  )

  # Update the experiment object
  experiment = conn.experiments(experiment.id).fetch()

# Fetch the best configuration and explore your experiment
all_best_assignments = conn.experiments(experiment.id).best_assignments().fetch()
# Returns a list of dict-like Observation objects
best_assignments = all_best_assignments.data[0].assignments
print("Best Assignments: " + str(best_assignments))
print("Explore your experiment: https://app.sigopt.com/experiment/" + experiment.id + "/analysis")

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9362  0.9340  0.9296  0.9444  0.9364  0.9361  0.0048  
MAE (testset)     0.7374  0.7396  0.7297  0.7448  0.7388  0.7380  0.0049  
Fit time          1.32    1.42    1.44    1.45    1.55    1.44    0.07    
Test time         0.17    0.31    0.19    0.32    0.24    0.24    0.06    
RMSE: 0.9390
0.9635456912349646
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
Created experiment: https://app.sigopt.com/experiment/1024554
RMSE: 0.9807
MAE:  0.7909
RMSE: 0.9664
MAE:  0.7751
RMSE: 0.9775
MAE:  0.7871
RMSE: 0.9720
MAE:  0.7807
RMSE: 0.9626
MAE:  0.7708
RMSE: 0.9647
MAE:  0.7736
RMSE: 0.9632
MAE:  0.7715
RMSE: 0.9723
MAE:  0.7814
RMSE: 0.9629
MAE:  0.7714
RMSE: 0.9730
MAE:  0.7826
RMSE: 0.9747
MAE:  0.7846
RMSE: 0.9786
MAE:  0.7881
RMSE: 0.9780
MAE:  0.7879
RMSE: 0.9625
MAE:  0.7707
RMSE: 0.9878
MAE:  0.7974
RMSE: 0.9727
MAE:  0.7826
RMSE: 0.962